In [ ]:
#gun-violence dataset import into notebook
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# Import the required libraries 
from plotly.offline import init_notebook_mode, iplot
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from wordcloud import WordCloud
from textblob import TextBlob 
import plotly.graph_objects as go
import seaborn as sns
import pandas as pd
import string, os, random
import calendar
from PIL import Image 
import numpy as np

from itertools import chain

import folium 
from folium import plugins 


init_notebook_mode(connected=True)
punc = string.punctuation
from datetime import datetime

# Load the dataset
gun_violence= pd.read_csv("../input/gun-violence-data/gun-violence-data_01-2013_03-2018.csv")


### Fix a row in the dataset 
### According to the author of this dataset, one particular incident is missing from the dataset 
### I have manually added this incident
### Related Thread: https://www.kaggle.com/jameslko/gun-violence-data/discussion/55307
missing_row = ['sban_1', '2017-10-01', 'Nevada', 'Las Vegas', 'Mandalay Bay 3950 Blvd S', 59, 489, 'https://en.wikipedia.org/wiki/2017_Las_Vegas_shooting', 'https://en.wikipedia.org/wiki/2017_Las_Vegas_shooting', '-', '-', '-', '-', '-', '36.095', 'Hotel', 
               '-115.171667', 47, 'Route 91 Harvest Festiva; concert, open fire from 32nd floor. 47 guns seized; TOTAL:59 kill, 489 inj, number shot TBD,girlfriend Marilou Danley POI', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
gun_violence.loc[len(gun_violence)] = missing_row

# Create some additional features
gun_violence['date'] = pd.to_datetime(gun_violence['date'])
gun_violence['year'] = gun_violence['date'].dt.year
gun_violence['month'] = gun_violence['date'].dt.month
gun_violence['monthday'] = gun_violence['date'].dt.day
gun_violence['weekday'] = gun_violence['date'].dt.weekday
gun_violence['loss'] = gun_violence['n_killed'] + gun_violence['n_injured']


In [ ]:
print("Gun Violence Dimensions:",gun_violence.shape)
gun_violence.head(10)

In [ ]:
#display last 4 elements in the dataset
gun_violence.tail(4)

In [ ]:
#check to see if there are spaces in column names
gun_violence.columns

In [ ]:
gun_violence.describe()

In [ ]:
#Useful information about the data
gun_violence.info()

In [ ]:
gun_violence['year'].describe()

In [ ]:
#  number of Gun Violence Incidents by year
# function to aggregate and return keys and values
def create_stack_bar_data(col):
    aggregated = gun_violence[col].value_counts()
    x_values = aggregated.index.tolist()
    y_values = aggregated.values.tolist()
    return x_values, y_values

x1, y1 = create_stack_bar_data('year')
x1 = x1[:-1]
y1 = y1[:-1]
trace1 = go.Bar(x=x1, y=y1, opacity=0.75, name="year count", marker=dict(color=['rgba(10, 220, 150, 0.6)', 'rgba(10, 220, 150, 0.6)', 'rgba(10, 220, 150, 0.6)', 'rgba(10, 220, 150, 0.6)', 'rgba(222,45,38,0.8)']))
layout = dict(height=400, title='Fig.1: Gun Violence Incidents by year', legend=dict(orientation="h"));
fig = go.Figure(data=[trace1], layout=layout);
iplot(fig);

In [ ]:
#Selecting Sample of the data that will be used (i.e. Months of 2018)
gun_violence_2017 = gun_violence[(gun_violence['year']==2017)]
print("Gun Violence Data Sample Dimension:", gun_violence.shape,"(i.e. Months of 2018)")#dataset dimension
gun_violence_2017.head()

In [ ]:
#Measures of the numerical data (i.e. Gun Violence in 2018)
gun_violence_2017.describe()

In [ ]:
gun_violence_2017.info()

**Above code isnt working

In [ ]:
extract =gun_violence_2017[['participant_gender','incident_id','participant_type','participant_age','participant_status','gun_stolen','gun_type','incident_characteristics']] 

extract['participant_type'] =  np.where(extract['participant_type']!='',extract['participant_type'],'NaN')
extract['participant_gender'] =  np.where(extract['participant_gender']!='',extract['participant_gender'],'NaN')
extract['participant_age'] =  np.where(extract['participant_age']!='',extract['participant_age'],'NaN')
extract['participant_status'] =  np.where(extract['participant_status']!='',extract['participant_status'],'NaN')
extract['gun_stolen'] =  np.where(extract['gun_stolen']!='',extract['gun_stolen'],'NaN')
extract['gun_type'] =  np.where(extract['gun_type']!='',extract['gun_type'],'NaN')
extract['incident_characteristics'] =  np.where(extract['gun_type']!='',extract['incident_characteristics'],'NaN')
#  removing special characters  regex
extract['participant_gender']=extract['participant_gender'].replace('[0-9||]', '',regex=True)
extract['participant_gender']=extract['participant_gender'].replace('[::]', ',',regex=True)

extract['participant_type']=extract['participant_type'].replace('[0-9||]', '',regex=True)
extract['participant_type']=extract['participant_type'].replace('[::]', ',',regex=True)

extract['participant_status']=extract['participant_status'].replace('[0-9||]', '',regex=True)
extract['participant_status']=extract['participant_status'].replace('[::]', ',',regex=True)

extract['participant_type']=extract['participant_type'].replace('[0-9||]', '',regex=True)
extract['participant_type']=extract['participant_type'].replace('[::]', ',',regex=True)

extract['gun_stolen']=extract['gun_stolen'].replace('[0-9||]', '',regex=True)
extract['gun_stolen']=extract['gun_stolen'].replace('[::]', ',',regex=True)

extract['gun_type']=extract['gun_type'].replace('[0-9||]', '',regex=True)
extract['gun_type']=extract['gun_type'].replace('[::]', ',',regex=True)

extract['incident_characteristics']=extract['incident_characteristics'].replace('[||]', ';',regex=True)


extract['participant_age']=extract['participant_age'].replace('[::]', ',',regex=True)
extract['participant_age']=extract['participant_age'].replace('[||]', ',',regex=True)
extract.head(10)


In [ ]:
extract = extract.dropna(axis=0)

extract.isnull().sum(axis=0)

In [ ]:
# convert to list
extract['participant_gender'] = extract['participant_gender'].str.split(',') 
extract['participant_type'] = extract['participant_type'].str.split(',') 
extract['participant_age'] = extract['participant_age'].str.split(',')
extract['participant_status'] = extract['participant_status'].str.split(',')
extract['gun_stolen'] = extract['gun_stolen'].str.split(',')
extract['gun_type'] = extract['gun_type'].str.split(',')
extract['incident_characteristics'] = extract['incident_characteristics'].str.split(';')



In [ ]:
extract.head(30)

In [ ]:
# convert list of pd.Series then stack it
gun_violence_gender = (extract
 .set_index(['incident_id'])['participant_gender']
 .apply(pd.Series)
 .stack()
 .reset_index()
 .rename(columns={0:'participant_gender'}))

gun_violence_type = (extract
 .set_index(['incident_id'])['participant_type']
 .apply(pd.Series)
 .stack()
 .reset_index()                  
 .rename(columns={0:'participant_type'}))








In [ ]:
gun_violence_age = (extract
 .set_index(['incident_id'])['participant_age']
 .apply(pd.Series)
 .stack()
 .reset_index()                 
 .rename(columns={0:'participant_age'}))


In [ ]:
gun_violence_participant_status = (extract
 .set_index(['incident_id'])['participant_status']
 .apply(pd.Series)
 .stack()
 .reset_index()                  
 .rename(columns={0:'participant_status'}))

gun_violence_gun_stolen = (extract
 .set_index(['incident_id'])['gun_stolen']
 .apply(pd.Series)
 .stack()
 .reset_index()                  
 .rename(columns={0:'gun_stolen'}))


In [ ]:
gun_violence_gun_type = (extract
 .set_index(['incident_id'])['gun_type']
 .apply(pd.Series)
 .stack()
 .reset_index()                   
 .rename(columns={0:'gun_type'}))

gun_violence_gun_incident_characteristics = (extract
 .set_index(['incident_id'])['incident_characteristics']
 .apply(pd.Series)
 .stack()
 .reset_index()                  
 .rename(columns={0:'incident_characteristics'}))

In [ ]:
gun_violence_age=gun_violence_age[gun_violence_age['participant_age'] != '']
gun_violence_age=gun_violence_age[gun_violence_age['participant_age'] != '-']
gun_violence_age=gun_violence_age[gun_violence_age['participant_age'].astype(int) > 6]
gun_violence_age

In [ ]:
gun_violence_gun_incident_characteristics

In [ ]:
# new_extract = pd.concat([gun_violence_gender, gun_violence_type,gun_violence_age,gun_violence_gun_stolen,gun_violence_participant_status,gun_violence_gun_type,gun_violence_gun_incident_characteristics],axis=1, join='inner')
# new_extract
 new_extract =  pd.merge(gun_violence_type,gun_violence_gender,how='inner', left_on=['incident_id','level_1'], right_on = ['incident_id','level_1'],)
 new_extract =  pd.merge(new_extract,gun_violence_age,how='inner', left_on=['incident_id','level_1'], right_on = ['incident_id','level_1'],)
 new_extract =  pd.merge(new_extract,gun_violence_gun_stolen,how='inner', left_on=['incident_id','level_1'], right_on = ['incident_id','level_1'],)
 new_extract =  pd.merge(new_extract,gun_violence_participant_status,how='inner', left_on=['incident_id','level_1'], right_on = ['incident_id','level_1'],)
 new_extract =  pd.merge(new_extract,gun_violence_gun_incident_characteristics,how='left', left_on=['incident_id','level_1'], right_on = ['incident_id','level_1'],)
 new_extract =  pd.merge(new_extract,gun_violence_gun_type,how='left', left_on=['incident_id','level_1'], right_on = ['incident_id','level_1'],)
    
# newextract = newextract.merge(gun_violence_gun_stolen,left_on='incident_id', right_on='incident_id', suffixes=(False, False))
# newextract = newextract.join(gun_violence_gun_type.set_index('incident_id'),how='inner')
# newextract = newextract.join(gun_violence_gun_incident_characteristics.set_index('incident_id'),how='inner')
# newextract = newextract[(newextract['participant_gender']!='')]
 new_extract

In [ ]:
new_extract['participant_type'] =  np.where(new_extract['participant_type']!='',new_extract['participant_type'],'Unidentified')
new_extract['participant_type'] =  np.where(new_extract['participant_type']!='-',new_extract['participant_type'],'Unidentified')

new_extract['participant_gender'] =  np.where(new_extract['participant_gender']!='',new_extract['participant_gender'],'Unidentified')
new_extract['participant_gender'] =  np.where(new_extract['participant_gender']!='-',new_extract['participant_gender'],'Unidentified')

new_extract

In [ ]:
new_extract = new_extract.dropna()
new_extract = new_extract.reset_index(drop=True)
new_extract

In [ ]:
Suspect = new_extract[new_extract['participant_type'] != 'Victim']
Suspect

In [ ]:
Suspectcount =Suspect['participant_gender'].value_counts()
Suspectcount

In [ ]:
#  number of Gun Violence Incidents by year
# function to aggregate and return keys and values
def create_stack_bar_data(col):
    aggregated = Suspect[col].value_counts()
    x_values = aggregated.index.tolist()
    y_values = aggregated.values.tolist()
    return x_values, y_values

x1, y1 = create_stack_bar_data('participant_gender')

trace = go.Pie(labels=x1, values=y1,title= 'Gender of Suspect')
iplot([trace], filename='basic_pie_chart')

In [ ]:
# 2014 time series
temp = gun_violence[gun_violence['year'] == 2014].groupby('date').agg({'state' : 'count', 'n_killed' : 'sum', 'n_injured' : 'sum'}).reset_index().rename(columns={'state' : 'incidents'})


trace1 = go.Scatter(x = temp.date, y = temp.incidents, name='Total Incidents', mode = "lines", marker = dict(color = '#c5d9f9'))
trace2 = go.Scatter(x = temp.date, y = temp.n_killed, name="Total Killed", mode = "lines", marker = dict(color = '#ff9f87'))
trace3 = go.Scatter(x = temp.date, y = temp.n_injured, name="Total Injured", mode = "lines", marker = dict(color = '#e8baff'))

data = [trace1, trace2, trace3]
layout = dict(height=350, title = 'Gun Violence Incidents - 2014', legend=dict(orientation="h", x=-.01, y=1), xaxis= dict(title='Date Time', ticklen= 1))
fig = dict(data = data, layout = layout)
iplot(fig)



# 2015 time series
temp = gun_violence[gun_violence['year'] == 2015].groupby('date').agg({'state' : 'count', 'n_killed' : 'sum', 'n_injured' : 'sum'}).reset_index().rename(columns={'state' : 'incidents'})


trace1 = go.Scatter(x = temp.date, y = temp.incidents, name='Total Incidents', mode = "lines", marker = dict(color = '#c5d9f9'))
trace2 = go.Scatter(x = temp.date, y = temp.n_killed, name="Total Killed", mode = "lines", marker = dict(color = '#ff9f87'))
trace3 = go.Scatter(x = temp.date, y = temp.n_injured, name="Total Injured", mode = "lines", marker = dict(color = '#e8baff'))

data = [trace1, trace2, trace3]
layout = dict(height=350, title = 'Gun Violence Incidents - 2015', legend=dict(orientation="h", x=-.01, y=1), xaxis= dict(title='Date Time', ticklen= 1))
fig = dict(data = data, layout = layout)
iplot(fig)





# 2016 time series
temp = gun_violence[gun_violence['year'] == 2016].groupby('date').agg({'state' : 'count', 'n_killed' : 'sum', 'n_injured' : 'sum'}).reset_index().rename(columns={'state' : 'incidents'})


trace1 = go.Scatter(x = temp.date, y = temp.incidents, name='Total Incidents', mode = "lines", marker = dict(color = '#c5d9f9'))
trace2 = go.Scatter(x = temp.date, y = temp.n_killed, name="Total Killed", mode = "lines", marker = dict(color = '#ff9f87'))
trace3 = go.Scatter(x = temp.date, y = temp.n_injured, name="Total Injured", mode = "lines", marker = dict(color = '#e8baff'))

data = [trace1, trace2, trace3]
layout = dict(height=350,title = 'Gun Violence Incidents - 2016', legend=dict(orientation="h", x=-.01, y=1), xaxis= dict(title='Date Time', ticklen= 1))
fig = dict(data = data, layout = layout)
iplot(fig)


# 2017 time series
temp = gun_violence[gun_violence['year'] == 2017].groupby('date').agg({'state' : 'count', 'n_killed' : 'sum', 'n_injured' : 'sum'}).reset_index().rename(columns={'state' : 'incidents'})


trace1 = go.Scatter(x = temp.date, y = temp.incidents, name='Total Incidents', mode = "lines", marker = dict(color = '#c5d9f9'))
trace2 = go.Scatter(x = temp.date, y = temp.n_killed, name="Total Killed", mode = "lines", marker = dict(color = '#ff9f87'))
trace3 = go.Scatter(x = temp.date, y = temp.n_injured, name="Total Injured", mode = "lines", marker = dict(color = '#e8baff'))

data = [trace1, trace2, trace3]
layout = dict(height=350,title = 'Gun Violence Incidents - 2017', legend=dict(orientation="h", x=-.01, y=1), xaxis= dict(title='Date Time', ticklen= 1))
fig = dict(data = data, layout = layout)
iplot(fig)

In [ ]:
states_df = gun_violence['state'].value_counts()

statesdf = pd.DataFrame()
statesdf['state'] = states_df.index
statesdf['counts'] = states_df.values

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

state_to_code = {'District of Columbia' : 'dc','Mississippi': 'MS', 'Oklahoma': 'OK', 'Delaware': 'DE', 'Minnesota': 'MN', 'Illinois': 'IL', 'Arkansas': 'AR', 'New Mexico': 'NM', 'Indiana': 'IN', 'Maryland': 'MD', 'Louisiana': 'LA', 'Idaho': 'ID', 'Wyoming': 'WY', 'Tennessee': 'TN', 'Arizona': 'AZ', 'Iowa': 'IA', 'Michigan': 'MI', 'Kansas': 'KS', 'Utah': 'UT', 'Virginia': 'VA', 'Oregon': 'OR', 'Connecticut': 'CT', 'Montana': 'MT', 'California': 'CA', 'Massachusetts': 'MA', 'West Virginia': 'WV', 'South Carolina': 'SC', 'New Hampshire': 'NH', 'Wisconsin': 'WI', 'Vermont': 'VT', 'Georgia': 'GA', 'North Dakota': 'ND', 'Pennsylvania': 'PA', 'Florida': 'FL', 'Alaska': 'AK', 'Kentucky': 'KY', 'Hawaii': 'HI', 'Nebraska': 'NE', 'Missouri': 'MO', 'Ohio': 'OH', 'Alabama': 'AL', 'Rhode Island': 'RI', 'South Dakota': 'SD', 'Colorado': 'CO', 'New Jersey': 'NJ', 'Washington': 'WA', 'North Carolina': 'NC', 'New York': 'NY', 'Texas': 'TX', 'Nevada': 'NV', 'Maine': 'ME'}
statesdf['state_code'] = statesdf['state'].apply(lambda x : state_to_code[x])

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = statesdf['state_code'],
        z = statesdf['counts'],
        locationmode = 'USA-states',
        text = statesdf['state'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Gun Violence Incidents")
        ) ]

layout = dict(
        title = 'State wise number of Gun Violence Incidents',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

# data = [ dict(
#         type='choropleth',
#         colorscale = scl,
#         autocolorscale = False,
#         locations = statesdf['state_code'],
#         z = statesdf['counts'],
#         locationmode = 'USA-states',
#         text = statesdf['state'],
#         marker = dict(
#             line = dict (
#                 color = 'rgb(255,255,255)',
#                 width = 2
#             ) ),
#         colorbar = dict(
#             title = "Gun Violence Incidents")
#         ) ]

# layout = dict(
#         title = 'State wise number of Gun Violence Incidents',
#         geo = dict(
#             scope='usa',
#             projection=dict( type='albers usa' ),
#             showlakes = True,
#             lakecolor = 'rgb(255, 255, 255)'),
#              )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

 Prominent Age of Gun Violence Suspects 



In [ ]:
def get_user_mapping(txt):
    if txt == "NA":
        return {}
    mapping = {}
    for d in txt.split("||"):
        try:
            key = d.split("::")[0]
            val = d.split("::")[1]
            if key not in mapping:
                mapping[key] = val
        except:
            pass
    return mapping

gun_violence['participant_type'] = gun_violence['participant_type'].fillna("NA")
gun_violence['participant_type_map'] = gun_violence['participant_type'].apply(lambda x : get_user_mapping(x))
gun_violence['participant_age'] = gun_violence['participant_age'].fillna("NA")
gun_violence['participant_age_map'] = gun_violence['participant_age'].apply(lambda x : get_user_mapping(x))
gun_violence['participant_gender'] = gun_violence['participant_gender'].fillna("NA")
gun_violence['participant_gender_map'] = gun_violence['participant_gender'].apply(lambda x : get_user_mapping(x))

## Finding the Suspect Age Groups
suspect_age_groups = {}
for i, row in gun_violence.iterrows():
    suspects = []
    for k,v in row['participant_type_map'].items():
        if "suspect" in v.lower():
            suspects.append(k)
    for suspect in suspects:
        if suspect in row['participant_age_map']:
            ag = row['participant_age_map'][suspect]
            if ag not in suspect_age_groups:
                suspect_age_groups[ag] = 0 
            else:
                suspect_age_groups[ag] += 1

trace1 = go.Bar(x=list(suspect_age_groups.keys()), y=list(suspect_age_groups.values()), opacity=0.75, name="month", marker=dict(color='rgba(200, 20, 160, 0.6)'))
layout = dict(height=400, title='Suspects Age - Distribution', xaxis=dict(range=[0, 100]), legend=dict(orientation="h"));
fig = go.Figure(data=[trace1], layout=layout)
iplot(fig)


 Prominent Age of Gun Violence Victims



In [ ]:
victim_age_groups = {}
for i, row in gun_violence.iterrows():
    victims = []
    for k,v in row['participant_type_map'].items():
        if "victim" in v.lower():
            victims.append(k)
    for victim in victims:
        if victim in row['participant_age_map']:
            ag = row['participant_age_map'][victim]
            if ag not in victim_age_groups:
                victim_age_groups[ag] = 0 
            else:
                victim_age_groups[ag] += 1
                
trace1 = go.Bar(x=list(victim_age_groups.keys()), y=list(victim_age_groups.values()), opacity=0.75, name="month", marker=dict(color='brown'))
layout = dict(height=400, title='Victims Age - Distribution', xaxis=dict(range=[0, 100]), legend=dict(orientation="h"));
fig = go.Figure(data=[trace1], layout=layout)
iplot(fig)

In [ ]:



statedate=gun_violence[['incident_id','state','city_or_county','monthday','weekday','loss']]
statedate


new_extract2 = pd.merge(statedate,new_extract,how='inner',left_on=['incident_id'], right_on = ['incident_id'],suffixes=(False,False))
new_extract2



In [ ]:

def create_label_encoder_dict(df):
    from sklearn.preprocessing import LabelEncoder
    
    label_encoder_dict = {}
    for column in df.columns:
        # Only create encoder for categorical data types
        if not np.issubdtype(df[column].dtype, np.number) and column != 'participant_age':
            label_encoder_dict[column]= LabelEncoder().fit(df[column])
    return label_encoder_dict

In [ ]:
data =new_extract2[['state','participant_type','participant_gender','participant_age','gun_stolen','participant_status','incident_characteristics','weekday']]
data

In [ ]:
def func(row):
    if row['weekday'] == 0:
        return 'Monday'
    elif row['weekday'] ==1:
        return 'Tuesday'
    elif row['weekday'] ==2:
        return 'Wednesday'
    elif row['weekday'] ==3:
        return 'Thursday'
    elif row['weekday'] ==4:
        return 'Friday'
    elif row['weekday'] ==5:
        return 'Saturday'
    elif row['weekday'] ==6:
        return 'Sunday'
    else:
        return 'Not a Vaild Date'

In [ ]:
data['weekday'] = data.apply(func, axis=1)
data

In [ ]:

label_encoders = create_label_encoder_dict(data)
print("Encoded Values for each Label")
print("="*32)
for column in label_encoders:
    print("="*32)
    print('Encoder(%s) = %s' % (column, label_encoders[column].classes_ ))
    print(pd.DataFrame([range(0,len(label_encoders[column].classes_))], columns=label_encoders[column].classes_, index=['Encoded Values']  ).T)

In [ ]:
 # Apply each encoder to the data set to obtain transformed values
data2 = data.copy() # create copy of initial data set
for column in data2.columns:
    if column in label_encoders:
        data2[column] = label_encoders[column].transform(data2[column])
        
print("Transformed data set")
print("="*32)
data2.head(15)


In [ ]:

# separate our data into dependent (Y) and independent(X) variables
X_data = data2[['state','participant_type','participant_gender','participant_age','gun_stolen','participant_status','incident_characteristics']]
Y_data = data2['weekday'] # actually weekday column

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.30)

In [ ]:
from sklearn.neural_network import MLPClassifier


In [ ]:
reg = MLPClassifier()

reg.fit(X_train,y_train)

In [ ]:
help(MLPClassifier)


In [ ]:
reg.n_layers_ # Number of layers utilized


In [ ]:
test_predicted = reg.predict(X_test)
test_predicted

In [ ]:
data3 = X_test.copy()
data3['predicted_weekday']=test_predicted
data3['predicted_weekday_en']=label_encoders['weekday'].inverse_transform(test_predicted)
data3['weekday']=data3['weekday']=y_test
data3['weekday_en']=label_encoders['weekday'].inverse_transform(y_test)
data3.head()

> **Evaluation**

Building a Confusion Matrix
NB. Data should be split in training and test data. The model built should be evaluated using unseen or test data

In [ ]:
k=(reg.predict(X_test) == y_test) # Determine how many were predicted correctly

k.value_counts()


In [ ]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test, reg.predict(X_test), labels=y_test.unique())
cm


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    import itertools
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plt.figure(figsize=(9,16))
plot_confusion_matrix(cm,data['weekday'].unique())

With Preprocessing


In [ ]:
def create_min_max_scaler_dict(df):
    from sklearn.preprocessing import MinMaxScaler
    min_max_scaler_dict = {}
    for column in df.columns:
        # Only create encoder for categorical data types
        if np.issubdtype(df[column].dtype, np.number):
            min_max_scaler_dict[column]= MinMaxScaler().fit(pd.DataFrame(df[column]))
    return min_max_scaler_dict

In [ ]:
scalerdata= data2.copy()

del scalerdata['weekday']
scalerdata


In [ ]:

min_max_scalers = create_min_max_scaler_dict(scalerdata)
print("Min Max Values for each Label")
print("="*32)
min_max_scalers

In [ ]:
#retrieving a scacler
state_scaler=min_max_scalers['state']

In [ ]:
state_scaler


In [ ]:
state_scaler.data_max_ 

In [ ]:
state_scaler.data_min_ 

In [ ]:
state_scaler.data_range_ 

In [ ]:
pd.DataFrame([
    {
        'column':col,
        'min':min_max_scalers[col].data_min_[0], 
        'max':min_max_scalers[col].data_max_[0], 
        'range':min_max_scalers[col].data_range_[0] }  for col in min_max_scalers])

In [ ]:
# Apply each scaler to the data set to obtain transformed values
data3 = data2.copy() # create copy of initial data set
for column in data3.columns:
    if column in min_max_scalers:
        data3[column] = min_max_scalers[column].transform(pd.DataFrame(data3[column]))

print("Transformed data set")
print("="*32)
data3.head(15)

In [ ]:
data3.describe()


In [ ]:
# separate our data into dependent (Y) and independent(X) variables
X2_data = data3[['state','participant_type','participant_gender','participant_age','gun_stolen','participant_status','incident_characteristics']]
Y2_data = data3['weekday'] # actually weekday column

In [ ]:
from sklearn.model_selection import train_test_split 
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_data, Y2_data, test_size=0.30)

In [ ]:
from sklearn.neural_network import MLPClassifier, MLPRegressor


In [ ]:
# Create an instance of linear regression
reg2 = MLPRegressor()
reg2.fit(X2_train,y2_train)

In [ ]:
reg2.n_layers_


In [ ]:
# Make predictions using the testing set
test2_predicted = reg2.predict(X2_test)
test2_predicted=test2_predicted.astype(int)

In [ ]:
data4 = X2_test.copy()
data4['predicted_weekday']=test2_predicted
data4['predicted_weekday_en']=label_encoders['weekday'].inverse_transform(test2_predicted)
data4['weekday']=data3['weekday']=y2_test
data4['weekday_en']=label_encoders['weekday'].inverse_transform(y2_test)
data4.head()

In [ ]:
k=(reg.predict(X2_test) == y2_test) # Determine how many were predicted correctly
k.value_counts()

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y2_test, reg.predict(X2_test), labels=y2_test.unique())

In [ ]:
plt.figure(figsize=(9,16))
plot_confusion_matrix(cm,data['weekday'].unique())

In [ ]:
#Decision tREE
treedata2 =  new_extract2[['city_or_county','participant_gender','participant_type','state','participant_age','weekday','incident_characteristics','gun_type']]
treedata2['incident_type'] = treedata2['incident_characteristics'].apply(lambda x: 'True' if x == 'Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)' else 'False')
treedata2

In [ ]:
def create_label_encoder_dict(df):
    from sklearn.preprocessing import LabelEncoder
    
    label_encoder_dict = {}
    for column in df.columns:
        # Only create encoder for categorical data types
        if not np.issubdtype(df[column].dtype, np.number):
            label_encoder_dict[column]= LabelEncoder().fit(df[column])
    return label_encoder_dict

In [ ]:
label_encoders = create_label_encoder_dict(treedata2)
print("Encoded Values for each Label")
print("="*32)
for column in label_encoders:
    print("="*32)
    print('Encoder(%s) = %s' % (column, label_encoders[column].classes_ ))
    print(pd.DataFrame([range(0,len(label_encoders[column].classes_))], columns=label_encoders[column].classes_, index=['Encoded Values']  ).T)

In [ ]:
# Apply each encoder to the data set to obtain transformed values
data5 = treedata2.copy() # create copy of initial data set
for column in data5.columns:
    if column in label_encoders:
        data5[column] = label_encoders[column].transform(data5[column])

print("Transformed data set")
print("="*32)
data5

In [ ]:
# separate our data into dependent (Y) and independent(X) variables
X_data = data5[['city_or_county','participant_type', 'participant_age','state','weekday','gun_type']]
Y_data = data5['incident_type']

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
clf = DecisionTreeClassifier(max_depth=2, criterion='entropy') # Change the max_depth to 10 or another number
clf.fit(X_data, Y_data)

In [ ]:
pd.DataFrame([ "%.2f%%" % perc for perc in (clf.feature_importances_ * 100) ], index = X_data.columns, columns = ['Feature Significance in Decision Tree'])


In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf,out_file=None, 
                                feature_names=X_data.columns, 
                         class_names=label_encoders[Y_data.name].classes_,  
                         filled=True, rounded=True,  proportion=True,
                                node_ids=True, #impurity=False,
                         special_characters=True)

In [ ]:
graph = graphviz.Source(dot_data) 
graph

In [ ]:
def tree_to_code(tree, feature_names, label_encoders={}):
    from sklearn.tree import _tree

    '''
    Outputs a decision tree model as a Python function
    
    Parameters:
    -----------
    tree: decision tree model
        The decision tree to represent as a function
    feature_names: list
        The feature names of the dataset used for building the decision tree
    '''

    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print ("def decision_tree({}):" .format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, threshold)) 
            recurse(tree_.children_right[node], depth + 1)
        else:
            #print(node)
            
            name = tree_.feature[node] 
            if name in label_encoders:
                if isinstance(label_encoders[name] , LabelEncoder) or True:
                    print ( "{}-return {}".format(indent, label_encoders[name].inverse_transform(tree_.value[node])))
                    return
            print ("{}return {} # Distribution of samples in node".format(indent, tree_.value[node]))

    recurse(0, 1)

In [ ]:
print("Decision Tree Rules")
print("="*32)
tree_to_code(clf, X_data.columns, label_encoders)

In [ ]:
print("Encoded Values for each Label")
print("="*32)
for column in label_encoders:
    print("="*32)
    print('Encoder(%s) = %s' % (column, label_encoders[column].classes_ ))
    print(pd.DataFrame([range(0,len(label_encoders[column].classes_))], columns=label_encoders[column].classes_, index=['Encoded Values']  ).T)

In [ ]:
k=(clf.predict(X_data) == Y_data) # Determine how many were predicted correctly
k.value_counts()


In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_data, clf.predict(X_data), labels=Y_data.unique())
cm

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    import itertools
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(cm,data5['incident_type'].unique())

In [ ]:

# separate our data into dependent (Y) and independent(X) variables
# separate our data into dependent (Y) and independent(X) variables
X3_data = data3[['weekday','participant_type','participant_gender','participant_age','gun_stolen','participant_status','incident_characteristics']]
Y3_data = data3['state'] # actually weekday column

In [ ]:
from sklearn.model_selection import train_test_split 
X3_train, X3_test, y3_train, y3_test = train_test_split(X3_data, Y3_data, test_size=0.30)

In [ ]:
# Import linear model package (has several regression classes)
from sklearn import linear_model

In [ ]:
# Create an instance of linear regression
reg = linear_model.LinearRegression()

In [ ]:
reg.fit(X_train,y_train)

In [ ]:
reg.coef_


In [ ]:
X_train.columns

In [ ]:
print("Regression Coefficients")
pd.DataFrame(reg.coef_,index=X_train.columns,columns=["Coefficient"])

In [ ]:
# Intercept
reg.intercept_

In [ ]:
# Make predictions using the testing set
test_predicted = reg.predict(X_test)
test_predicted

In [ ]:

data6 = X_test.copy()
data6['predicted_state_level']=test_predicted
data6['state_level']=y_test
data6.head()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, test_predicted))

In [ ]:
print('Variance score: %.2f' % r2_score(y_test, test_predicted))

In [ ]:
help(reg.score)


In [ ]:
reg.score(X_test,y_test)


In [ ]:
from sklearn.decomposition import PCA


In [ ]:
pca = PCA(n_components=1)


In [ ]:
pca.fit(data2[X3_train.columns])


In [ ]:
pca.components_


In [ ]:
pca.n_features_


In [ ]:
pca.n_components_


In [ ]:
X3_test


In [ ]:
X3_test =X3_test.fillna(X3_test.mean())
X_reduced = pca.transform(X3_test)
X_reduced

In [ ]:
plt.scatter(X_reduced, y3_test,  color='black')


In [ ]:

plt.scatter(X_reduced, y3_test,  color='black')
plt.plot(X_reduced, test_predicted, color='blue',linewidth=1)
plt.plot(X_reduced, test_predicted, color='red',linewidth=1)

#plt.xticks(())
#plt.yticks(())

plt.show()

In [ ]:
plt.plot(y3_test, test_predicted, 'ro-')


In [ ]:
np.std(np.abs(y3_test-test_predicted))


In [ ]:
plt.plot(y3_test, test_predicted, 'ro-')


In [ ]:
np.std(np.abs(y_test-test_predicted))


In [ ]:
data4=pd.DataFrame({'actual':y_test,'pred':test_predicted})
data4.head()

In [ ]:
data4.sort_values('actual').plot(kind='line',x='actual',y='pred')


In [ ]:
data.corr()